In [66]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os 
pd.set_option('display.max_columns', None)

Dans le cadre du data challenge QRT de cette année, nous vous proposons un challenge de prédiction de résultat de match. Vous recevrez des données historiques réelles au niveau des équipes et des joueurs, et vous devrez prédire quelle équipe gagne ou s'il y a un match nul.

Le score cible est l’accuracy de la prédiction pour les trois classes [HOME_WINS, DRAW, AWAY_WINS], Il existe donc pour un match trois outputs possibles, [1,0,0]. [0,1,0] et [0,0,1].

In [72]:
lien = '/Users/augustincablant/Desktop/QRT/train'
liste_df = []
for dirname, _, filenames in os.walk(lien):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        liste_df.append(pd.read_csv(lien + '/' + filename))

player_away, player_home, team_away, team_home, y_train = liste_df[0], liste_df[1], liste_df[2], liste_df[4], liste_df[3]

for col in player_away.columns:
    if col not in player_home.columns:
        print(col)

for col in team_away.columns:
    if col not in team_home.columns:
        print(col)

#Ces deux datasets ont les mêmes colonnes. On créer donc la colonne home qui vaut True ou False et on agrège les deux datasets !

player_away['HOME'], player_home['HOME'] = False, True
df_player = pd.concat([player_home, player_away], ignore_index=True).sample(frac=1).reset_index(drop=True)

team_away['HOME'], team_home['HOME'] = False, True
df_team = pd.concat([team_away, team_home], ignore_index=True).sample(frac=1).reset_index(drop=True)


##test 

lien = '/Users/augustincablant/Desktop/QRT/test'
liste_df = []
for dirname, _, filenames in os.walk(lien):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        liste_df.append(pd.read_csv(lien + '/' + filename))

team_away, player_home, team_home, player_away = liste_df[0], liste_df[1], liste_df[2], liste_df[3]

player_away['HOME'], player_home['HOME'] = False, True
df_player_test = pd.concat([player_home, player_away], ignore_index=True).sample(frac=1).reset_index(drop=True)

team_away['HOME'], team_home['HOME'] = False, True
df_team_test = pd.concat([team_away, team_home], ignore_index=True).sample(frac=1).reset_index(drop=True)

/Users/augustincablant/Desktop/QRT/train/train_away_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/train_home_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/train_away_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/Y_train.csv
/Users/augustincablant/Desktop/QRT/train/train_home_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_away_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_home_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_home_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_away_player_statistics_df.csv


## Idée numéro 1 

On construit un dataset sur le dataset df_team. 
Pour évalue le niveau de chaque joueur de chaque team grâce à une métrique que l'on construit, puis on fait une moyenne sur l'ensemble des joueurs de l'équipe.

In [73]:
teams = df_team['TEAM_NAME'].unique()
df_team

for team in teams:
    sub = df_player[df_player['TEAM_NAME']==team]
    sub_numeric = sub.drop(columns=['LEAGUE', 'TEAM_NAME', 'POSITION', 'ID', 'PLAYER_NAME', 'HOME'])
    mean = sub_numeric.mean(axis=1, skipna=True).mean()
    df_team


KeyError: 'TEAM_NAME'

In [74]:
sns.heatmap(df_team)

,ID,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,TEAM_CORNERS_season_sum,TEAM_FOULS_season_sum,TEAM_YELLOWCARDS_season_sum,TEAM_REDCARDS_season_sum,TEAM_OFFSIDES_season_sum,TEAM_ATTACKS_season_sum,TEAM_PENALTIES_season_sum,TEAM_SUBSTITUTIONS_season_sum,TEAM_BALL_SAFE_season_sum,TEAM_DANGEROUS_ATTACKS_season_sum,TEAM_INJURIES_season_sum,TEAM_GOALS_season_sum,TEAM_GAME_WON_season_sum,TEAM_GAME_DRAW_season_sum,TEAM_GAME_LOST_season_sum,TEAM_SHOTS_TOTAL_season_average,TEAM_SHOTS_INSIDEBOX_season_average,TEAM_SHOTS_OFF_TARGET_season_average,TEAM_SHOTS_ON_TARGET_season_average,TEAM_SHOTS_OUTSIDEBOX_season_average,TEAM_PASSES_season_average,TEAM_SUCCESSFUL_PASSES_season_average,TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average,TEAM_SAVES_season_average,TEAM_CORNERS_season_average,TEAM_BALL_POSSESSION_season_average,TEAM_FOULS_season_average,TEAM_YELLOWCARDS_season_average,TEAM_REDCARDS_season_average,TEAM_OFFSIDES_season_average,TEAM_ATTACKS_season_average,TEAM_PENALTIES_season_average,TEAM_SUBSTITUTIONS_season_average,TEAM_BALL_SAFE_season_average,TEAM_DANGEROUS_ATTACKS_season_average,TEAM_INJURIES_season_average,TEAM_GOALS_season_average,TEAM_GAME_WON_season_average,TEAM_GAME_DRAW_season_average,TEAM_GAME_LOST_season_average,TEAM_SHOTS_TOTAL_season_std,TEAM_SHOTS_INSIDEBOX_season_std,TEAM_SHOTS_OFF_TARGET_season_std,TEAM_SHOTS_ON_TARGET_season_std,TEAM_SHOTS_OUTSIDEBOX_season_std,TEAM_PASSES_season_std,TEAM_SUCCESSFUL_PASSES_season_std,TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std,TEAM_SAVES_season_std,TEAM_CORNERS_season_std,TEAM_BALL_POSSESSION_season_std,TEAM_FOULS_season_std,TEAM_YELLOWCARDS_season_std,TEAM_REDCARDS_season_std,TEAM_OFFSIDES_season_std,TEAM_ATTACKS_season_std,TEAM_PENALTIES_season_std,TEAM_SUBSTITUTIONS_season_std,TEAM_BALL_SAFE_season_std,TEAM_DANGEROUS_ATTACKS_season_std,TEAM_INJURIES_season_std,TEAM_GOALS_season_std,TEAM_SHOTS_TOTAL_5_last_match_sum,TEAM_SHOTS_INSIDEBOX_5_last_match_sum,TEAM_SHOTS_OFF_TARGET_5_last_match_sum,TEAM_SHOTS_ON_TARGET_5_last_match_sum,TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum,TEAM_PASSES_5_last_match_sum,TEAM_SUCCESSFUL_PASSES_5_last_match_sum,TEAM_SAVES_5_last_match_sum,TEAM_CORNERS_5_last_match_sum,TEAM_FOULS_5_last_match_sum,TEAM_YELLOWCARDS_5_last_match_sum,TEAM_REDCARDS_5_last_match_sum,TEAM_OFFSIDES_5_last_match_sum,TEAM_ATTACKS_5_last_match_sum,TEAM_PENALTIES_5_last_match_sum,TEAM_SUBSTITUTIONS_5_last_match_sum,TEAM_BALL_SAFE_5_last_match_sum,TEAM_DANGEROUS_ATTACKS_5_last_match_sum,TEAM_INJURIES_5_last_match_sum,TEAM_GOALS_5_last_match_sum,TEAM_GAME_WON_5_last_match_sum,TEAM_GAME_DRAW_5_last_match_sum,TEAM_GAME_LOST_5_last_match_sum,TEAM_SHOTS_TOTAL_5_last_match_average,TEAM_SHOTS_INSIDEBOX_5_last_match_average,TEAM_SHOTS_OFF_TARGET_5_last_match_average,TEAM_SHOTS_ON_TARGET_5_last_match_average,TEAM_SHOTS_OUTSIDEBOX_5_last_match_average,TEAM_PASSES_5_last_match_average,TEAM_SUCCESSFUL_PASSES_5_last_match_average,TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,TEAM_SAVES_5_last_match_average,TEAM_CORNERS_5_last_match_average,TEAM_BALL_POSSESSION_5_last_match_average,TEAM_FOULS_5_last_match_average,TEAM_YELLOWCARDS_5_last_match_average,TEAM_REDCARDS_5_last_match_average,TEAM_OFFSIDES_5_last_match_average,TEAM_ATTACKS_5_last_match_average,TEAM_PENALTIES_5_last_match_average,TEAM_SUBSTITUTIONS_5_last_match_average,TEAM_BALL_SAFE_5_last_match_average,TEAM_DANGEROUS_ATTACKS_5_last_match_average,TEAM_INJURIES_5_last_match_average,TEAM_GOALS_5_last_match_average,TEAM_GAME_WON_5_last_match_average,TEAM_GAME_DRAW_5_last_match_average,TEAM_GAME_LOST_5_last_match_average,TEAM_SHOTS_TOTAL_5_last_match_std,TEAM_SHOTS_INSIDEBOX_5_last_match_std,TEAM_SHOTS_OFF_TARGET_5_last_match_std,TEAM_SHOTS_ON_TARGET_5_last_match_std,TEAM_SHOTS_OUTSIDEBOX_5_last_match_std,TEAM_PASSES_5_last_match_std,TEAM_SUCCESS

In [75]:
df_player_test

,ID,POSITION,PLAYER_ACCURATE_CROSSES_season_sum,PLAYER_ACCURATE_PASSES_season_sum,PLAYER_AERIALS_WON_season_sum,PLAYER_ASSISTS_season_sum,PLAYER_BIG_CHANCES_CREATED_season_sum,PLAYER_BIG_CHANCES_MISSED_season_sum,PLAYER_BLOCKED_SHOTS_season_sum,PLAYER_CAPTAIN_season_sum,PLAYER_CLEARANCES_season_sum,PLAYER_CLEARANCE_OFFLINE_season_sum,PLAYER_DISPOSSESSED_season_sum,PLAYER_DRIBBLED_ATTEMPTS_season_sum,PLAYER_DRIBBLED_PAST_season_sum,PLAYER_DUELS_LOST_season_sum,PLAYER_DUELS_WON_season_sum,PLAYER_ERROR_LEAD_TO_GOAL_season_sum,PLAYER_FOULS_season_sum,PLAYER_FOULS_DRAWN_season_sum,PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum,PLAYER_GOALS_season_sum,PLAYER_GOALS_CONCEDED_season_sum,PLAYER_HIT_WOODWORK_season_sum,PLAYER_INTERCEPTIONS_season_sum,PLAYER_KEY_PASSES_season_sum,PLAYER_MINUTES_PLAYED_season_sum,PLAYER_OFFSIDES_season_sum,PLAYER_OWN_GOALS_season_sum,PLAYER_PASSES_season_sum,PLAYER_PENALTIES_COMMITTED_season_sum,PLAYER_PENALTIES_MISSES_season_sum,PLAYER_PENALTIES_SAVED_season_sum,PLAYER_PENALTIES_SCORED_season_sum,PLAYER_PENALTIES_WON_season_sum,PLAYER_REDCARDS_season_sum,PLAYER_SAVES_season_sum,PLAYER_SAVES_INSIDE_BOX_season_sum,PLAYER_SHOTS_BLOCKED_season_sum,PLAYER_SHOTS_ON_TARGET_season_sum,PLAYER_SHOTS_TOTAL_season_sum,PLAYER_STARTING_LINEUP_season_sum,PLAYER_SUCCESSFUL_DRIBBLES_season_sum,PLAYER_TACKLES_season_sum,PLAYER_TOTAL_CROSSES_season_sum,PLAYER_TOTAL_DUELS_season_sum,PLAYER_YELLOWCARDS_season_sum,PLAYER_PUNCHES_season_sum,PLAYER_LONG_BALLS_season_sum,PLAYER_LONG_BALLS_WON_season_sum,PLAYER_SHOTS_OFF_TARGET_season_sum,PLAYER_ACCURATE_CROSSES_season_average,PLAYER_ACCURATE_PASSES_season_average,PLAYER_ACCURATE_PASSES_PERCENTAGE_season_average,PLAYER_AERIALS_WON_season_average,PLAYER_ASSISTS_season_average,PLAYER_BIG_CHANCES_CREATED_season_average,PLAYER_BIG_CHANCES_MISSED_season_average,PLAYER_BLOCKED_SHOTS_season_average,PLAYER_CAPTAIN_season_average,PLAYER_CLEARANCES_season_average,PLAYER_CLEARANCE_OFFLINE_season_average,PLAYER_DISPOSSESSED_season_average,PLAYER_DRIBBLED_ATTEMPTS_season_average,PLAYER_DRIBBLED_PAST_season_average,PLAYER_DUELS_LOST_season_average,PLAYER_DUELS_WON_season_average,PLAYER_ERROR_LEAD_TO_GOAL_season_average,PLAYER_FOULS_season_average,PLAYER_FOULS_DRAWN_season_average,PLAYER_GOALKEEPER_GOALS_CONCEDED_season_average,PLAYER_GOALS_season_average,PLAYER_GOALS_CONCEDED_season_average,PLAYER_HIT_WOODWORK_season_average,PLAYER_INTERCEPTIONS_season_average,PLAYER_KEY_PASSES_season_average,PLAYER_MINUTES_PLAYED_season_average,PLAYER_OFFSIDES_season_average,PLAYER_OWN_GOALS_season_average,PLAYER_PASSES_season_average,PLAYER_PENALTIES_COMMITTED_season_average,PLAYER_PENALTIES_MISSES_season_average,PLAYER_PENALTIES_SAVED_season_average,PLAYER_PENALTIES_SCORED_season_average,PLAYER_PENALTIES_WON_season_average,PLAYER_RATING_season_average,PLAYER_REDCARDS_season_average,PLAYER_SAVES_season_average,PLAYER_SAVES_INSIDE_BOX_season_average,PLAYER_SHOTS_BLOCKED_season_average,PLAYER_SHOTS_ON_TARGET_season_average,PLAYER_SHOTS_TOTAL_season_average,PLAYER_STARTING_LINEUP_season_average,PLAYER_SUCCESSFUL_DRIBBLES_season_average,PLAYER_TACKLES_season_average,PLAYER_TOTAL_CROSSES_season_average,PLAYER_TOTAL_DUELS_season_average,PLAYER_YELLOWCARDS_season_average,PLAYER_PUNCHES_season_average,PLAYER_LONG_BALLS_season_average,PLAYER_LONG_BALLS_WON_season_average,PLAYER_SHOTS_OFF_TARGET_season_average,PLAYER_ACCURATE_CROSSES_season_std,PLAYER_ACCURATE_PASSES_season_std,PLAYER_ACCURATE_PASSES_PERCENTAGE_season_std,PLAYER_AERIALS_WON_season_std,PLAYER_ASSISTS_season_std,PLAYER_BIG_CHANCES_CREATED_season_std,PLAYER_BIG_CHANCES_MISSED_season_std,PLAYER_BLOCKED_SHOTS_season_std,PLAYER_CAPTAIN_season_std,PLAYER_CLEARANCES_season_std,PLAYER_CLEARANCE_OFFLINE_season_std,PLAYER_DISPOSSESSED_season_std,PLAYER_DRIBBLED_ATTEMPTS_season_std,PLAYER_DRIBBLED_PAST_season_std,PLAYER_DUELS_LOST_season_std,PLAYER_DUELS_WON_season_std,PLAYER_ERROR_LEAD_TO_GOAL_season_std,PLAYER_FOULS_season_std,PLAYER_FOULS_DRAWN_season_std,PLAYER_G